# Introduction

The aim of the following notebook is to make the user comfortable with utilizing the LUT(lookup table) and the accompanying `look_up` function for their transistor sizing flow.

We try various plots which we would conventionally plot in Cadence, not for purposes of parameter extraction but usually to verify the sanity of MOSFET models. For example, first order effects like verifying square law, drain current saturation. Then the second order effects like gm saturation, body effect and channel length modulation.

Additionally, the key intent of this is exercise is to show the readers that the lookup tables are generated from the actual transistor models by Spectre. So it is encouraged that you plot a certain plot for a given voltages/length and try to compare it with actual value from your simulation. This will increase your confidence in the lookup table as well as provide another layer of sanity check. Additionally, this could serve as a demonstration of limitations of the extrapolation and could give you a guideline in how to select appropriate step sizes for future lookup table generation.

Each plot is followed by an empty observation sections, where I encourage you to plot and make observations for future reference.

The following code block essentially imports/includes the necessary libraries. Additionally, it tries to import the config file named `conf.py`. You defined the values in this file in [Import Demo](./import_demo.ipynb). Do not attempt to write this file manually. If you want to rewrite head back to the relevant section of [Import Demo](./import_demo.ipynb). Make sure to append the correct library path to the system path. In case you are using the `docs/notebooks` directory `../../` is your library path.

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
# Append the path of your library to the system path
sys.path.append(os.path.abspath("../../"))
from analog_daddy.look_up import look_up
from analog_daddy.utils import pretty_print_structure, describe_structure
from analog_daddy.conf import * # import all the config variables

In [ ]:
z = np.load('../../sample_files/GPDK45.npy', allow_pickle=True).item()
nmos = z['nmos_svt']
pmos = z['pmos_svt']

## Understanding the structure of the Lookup table.

While the `look_up` function abstracts all the process of accessing the LUT. It is essential to understand the structure of the LUT. The `describe_structure` function does that and `pretty_print_structure` does what it says.

In [ ]:
pretty_print_structure(describe_structure(nmos))

What does the above data say? Firstly, it shows the first two and the last two elements of the "ds", "gs", "length" and "sb" array. (Or whatever your keys are). This is a one dimensional numpy array. Secondly, at the last you will see the width. Finally, you can see the shape of the 4D numpy array for parameters like "cgg", "gm" et cetera. But where is the Corner, Temperature and description? It is in the parent dictionary `z` which was imported using `np.load`.

In [ ]:
describe_structure(z)

But how are the 4D arrays structured? They are structured as ds, gs, length and sb. So to access 'cgg' you can access
it using `nmos['cgg'][ds_index][gs_index][length_index][sb_index]`. Hopefully you don't need to do this, but keeping it here just in case you are curious.

## Sample plots for practice.

The aim of the following tasks is to provide ready to use templates for common plots. They will help you understand how the lookup table (LUT) or `.npy` can be used to generate plots, format them and obtain any of the device's saved DC operating point parameters. When we don't specify any width and request a quantity which depends on Width (`id` or `cgg` for example)
it returns the result for the simulated width which is default width. To know this width you can use `nmos['w']` or `pmos['w']`. For any other widths the parameters can be scaled linearly.

In [ ]:
# Sweep voltage across supply. We are just creating an array for future use.
v_sweep = np.arange(0, VDD, VOLTAGE_STEP_SIZE)
l_sweep = np.arange(L_MIN, L_MAX, 0.5e-6) # 0.5um steps. 

## Plotting the $I_d$ vs $V_{gs}$ characteristics of MOSFET.

In [ ]:
# The following one liner does the following. It looks up the value of id for the given
# vgs (here it is a an array from 0 to VDD in steps of VOLTAGE_STEP_SIZE),
# a length of 1um, the drain source voltage of VDD and a source bulk voltage of 0.
id_nmos = look_up(nmos, 'id', length=1e-06, gs=v_sweep, ds=VDD, sb=0)
id_pmos = look_up(pmos, 'id', length=1e-06, gs=v_sweep, ds=VDD, sb=0)


fig = go.Figure()
# while plotting we are dividing the obtained value by 1e-06 to represent it in microamps. Additionally, we are
# taking the absolute value of the pmos current since it is negative. This should not be needed if the expressions
# are defined correctly when importing the expression to ADE assembler.
# NMOS plot
fig.add_trace(go.Scatter(x=v_sweep, y=id_nmos/1e-06, mode='lines', name='NMOS', line=dict(color='red')))

# PMOS plot
fig.add_trace(go.Scatter(x=v_sweep, y=abs(id_pmos)/1e-06, mode='lines', name='PMOS', line=dict(color='magenta')))

fig.update_layout(
    title_text=r'$I_d vs V_{gs} plot for NMOS and PMOS$',
    xaxis_title=r'$V_{gs}$ [V]',
    yaxis_title=r'$I_d$ [$\mu$A]'
)
fig.show()

### Observations


## Plotting the $I_d$ vs $V_{ds}$ characteristics of MOSFET.

In [ ]:
# The following one liner does the following. It looks up the value of id for the given
# vds (here it is a an array from 0 to VDD in steps of VOLTAGE_STEP_SIZE),
# a length of 1um, the gate source voltage of 0.8 and a source bulk voltage of 0.
id_nmos = look_up(nmos, 'id', length=1e-06, ds=v_sweep, gs=0.8, sb=0)
id_pmos = look_up(pmos, 'id', length=1e-06, ds=v_sweep, gs=0.8, sb=0)


fig = go.Figure()
# while plotting we are dividing the obtained value by 1e-06 to represent it in microamps. Additionally, we are
# taking the absolute value of the pmos current since it is negative. This should not be needed if the expressions
# are defined correctly when importing the expression to ADE assembler.
# NMOS plot
fig.add_trace(go.Scatter(x=v_sweep, y=id_nmos/1e-06, mode='lines', name='NMOS', line=dict(color='red')))

# PMOS plot
fig.add_trace(go.Scatter(x=v_sweep, y=abs(id_pmos)/1e-06, mode='lines', name='PMOS', line=dict(color='magenta')))

fig.update_layout(
    title_text=r'$I_d vs V_{ds} plot for NMOS and PMOS$',
    xaxis_title=r'$V_{ds}$ [V]',
    yaxis_title=r'$I_d$ [$\mu$A]'
)
fig.show()

### Observations

Can you see the saturation of the drain current?

## Multidimensional Lookups.

So the `look_up` function currently does not support multidimensional lookups. However that should not stop us right now.
The way we will do it is to iterate lookup along the parameter of interest. The following examples shows how to do so.

## Plotting the $I_d$ vs $V_{gs}$ characteristics of MOSFET for different $V_{ds}$

In [ ]:
# The following one liner does the following. It looks up the value of id for the given
# vgs (here it is a an array from 0 to VDD in steps of VOLTAGE_STEP_SIZE),
# a length of 1um, the six linearly spaced drain source voltages from 0 to VDD and a source bulk voltage of 0.
param_sweep = np.linspace(0, VDD, 6) # your parameter array.
id_nmos = np.array([look_up(nmos, 'id', length=1e-06, ds=ds, gs=v_sweep, sb=0) for ds in param_sweep])
id_pmos = np.array([look_up(pmos, 'id', length=1e-06, ds=ds, gs=v_sweep, sb=0) for ds in param_sweep])
fig = go.Figure()

# currently i am using the brute force way of appending to existing figure.
# I will update this to a better way in the future where it takes mxn array directly.
for idx, param in enumerate(param_sweep):
    fig.add_trace(go.Scatter(x=v_sweep, y=id_nmos[idx], mode='lines', name=f'Vds={param} V'))

fig.update_layout(title='Parametric Plot of Id vs Vgs for different Vds', xaxis_title='Vgs', yaxis_title='Id')
fig.show()


### Observations

## Plotting the $I_d$ vs $V_{ds}$ characteristics of MOSFET for different $V_{gs}$

In [ ]:
# The following one liner does the following. It looks up the value of id for the given
# vds (here it is a an array from 0 to VDD in steps of VOLTAGE_STEP_SIZE),
# a length of 1um, the six linearly spaced gate source voltages from 0 to VDD and a source bulk voltage of 0.
param_sweep = np.linspace(0, VDD, 6) # your parameter array.
id_nmos = np.array([look_up(nmos, 'id', length=1e-06, gs=gs, ds=v_sweep, sb=0) for gs in param_sweep])
id_pmos = np.array([look_up(pmos, 'id', length=1e-06, gs=gs, ds=v_sweep, sb=0) for gs in param_sweep])
fig = go.Figure()

for idx, param in enumerate(param_sweep):
    fig.add_trace(go.Scatter(x=v_sweep, y=id_nmos[idx], mode='lines', name=f'Vgs={param} V'))

fig.update_layout(title='Parametric Plot of Id vs Vds for different Vgs', xaxis_title='Vds', yaxis_title='Id')
fig.show()


### Observations

## Plotting the $g_m$ vs $V_{gs}$ characteristics of MOSFET.

In [ ]:
# The following one liner does the following. It looks up the value of gm for the given
# vgs (here it is a an array from 0 to VDD in steps of VOLTAGE_STEP_SIZE),
# a length of 1um, the drain source voltage of VDD and a source bulk voltage of 0.
gm_nmos = look_up(nmos, 'gm', length=1e-06, gs=v_sweep, ds=VDD, sb=0)
gm_pmos = look_up(pmos, 'gm', length=1e-06, gs=v_sweep, ds=VDD, sb=0)


fig = go.Figure()

fig.add_trace(go.Scatter(x=v_sweep, y=gm_nmos, mode='lines', name='NMOS', line=dict(color='red')))

# PMOS plot
fig.add_trace(go.Scatter(x=v_sweep, y=gm_pmos, mode='lines', name='PMOS', line=dict(color='magenta')))

fig.update_layout(
    title_text=r'$g_m vs V_{gs} plot for NMOS and PMOS$',
    xaxis_title=r'$V_{gs}$ [V]',
    yaxis_title=r'$g_m$ [A/V]'
)
fig.show()

### Observations

- Do you observe that gm saturates with increasing vgs.

## Plotting the $g_m\over i_d$ vs $V_{gs}$ characteristics of MOSFET.

In [ ]:
# The following one liner does the following. It looks up the value of gm for the given
# vgs (here it is a an array from 0 to VDD in steps of VOLTAGE_STEP_SIZE),
# a length of 1um, the drain source voltage of VDD and a source bulk voltage of 0.
gm_id_nmos = look_up(nmos, 'gm_id', length=1e-06, gs=v_sweep, ds=VDD, sb=0)
gm_id_pmos = look_up(pmos, 'gm_id', length=1e-06, gs=v_sweep, ds=VDD, sb=0)


fig = go.Figure()

fig.add_trace(go.Scatter(x=v_sweep, y=gm_id_nmos, mode='lines', name='NMOS', line=dict(color='red')))

# PMOS plot
fig.add_trace(go.Scatter(x=v_sweep, y=abs(gm_id_pmos), mode='lines', name='PMOS', line=dict(color='magenta')))

fig.update_layout(
    title_text=r'$g_m vs V_{gs} plot for NMOS and PMOS$',
    xaxis_title=r'$V_{gs}$ [V]',
    yaxis_title=r'$g_m$ [A/V]'
)
fig.show()

### Observations

TODO: Add limits

## Comparison between piecewise linear model and actual $g_m$

### Observations

## Variation of Threshold Voltage due to body effect.

### Observations

## $g_m$ vs $V_{gs}$ for different lengths, and same $\beta$

### Observations